In [1]:
import numpy as np
import pandas as ps
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
from harp_beam import compute_EEPs
from functions import to_dBV
from plots import plot2

In [2]:
## Q 2. plot all the 256 EEPs and their average (AEP)
num_dir = 256
theta = np.linspace(-np.pi/2, np.pi/2, num_dir)
phi = np.zeros_like(theta)

# Compute EEPs
v_theta_polY, v_phi_polY, v_theta_polX, v_phi_polX = compute_EEPs(theta.copy()[:, None], phi.copy()[:, None])

# Calculate magnitude of EEPs in dBV
magnitude_EEP_polY = to_dBV(np.abs(v_theta_polY))
magnitude_EEP_polX = to_dBV(np.abs(v_theta_polX))

# Calculate AEPs
AEP_polY = to_dBV(np.sqrt(np.mean(np.abs(v_theta_polY)**2, axis=1)))
AEP_polX = to_dBV(np.sqrt(np.mean(np.abs(v_theta_polX)**2, axis=1)))

# plot EEPs and AEPs
#plot2(theta, v_theta_polY, v_theta_polX, magnitude_EEP_polY, magnitude_EEP_polX, AEP_polY, AEP_polX)

In [3]:
## Q 3. the model matrix, the covariance matrix, the exact gain values and (my) gain estimations are loaded as
filename_vismat = f"data_20feb2024_2330_100MHz.mat"
mat = scipy.io.loadmat(filename_vismat)
R = np.array(mat['R']) # covariance matrix
M_AEP = np.array(mat['M_AEP']) # model matrix using AEP
M_EEPs = np.array(mat['M_EEPs']) # model matrix using all EEPs
g_sol = np.array(mat['g_sol']) # exact gain solution
g_AEP = np.array(mat['g_AEP']) # estimation using M_AEP (using this for question 5 and 6 only if you haven't been able to complete question 3 and 4)
g_EEPs = np.array(mat['g_EEPs']) # estimation using M_EEPs

In [39]:
# implement StEFCal

max_iteration = 100
threshold = 1e-8

# initialise identity matrix for the gain matrix
G = np.eye(len(g_sol))

# iterative loop for i
for i in range(max_iteration):
    G_prev = G.copy()  # You need to copy G from the last iteration for comparison later

    # Iterative loop for antennas p
    for p in range(G.shape[0]):
        z = np.dot(G_prev, M_AEP[:, p])  # Use all rows of M_AEP for antenna p
        gp = np.dot(np.conjugate(R[:, p]).T, z) / np.dot(np.conjugate(z).T, z)  # Calculate new gain for antenna p
        G[p, p] = gp  # Update the gain for antenna p

    # Convergence check
    if np.linalg.norm(G - G_prev, 'fro') / np.linalg.norm(G, 'fro') < threshold:
        print(f"Convergence reached after {i+1} iterations.")
        break
    elif i % 2 == 0:
        # Averaging step
        G = (G + G_prev) / 2


/var/folders/2g/9t7c77957pl9vb9x2t79vs2c0000gn/T/ipykernel_26158/1678686017.py:17: ComplexWarning: Casting complex values to real discards the imaginary part
  G[p, p] = gp  # Update the gain for antenna p


Convergence reached after 61 iterations.


In [40]:
G

array([[1.20307849, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.13905793, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.4298976 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.73081644, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.83648685,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.48974225]])